In [1]:

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='.env')
openai_api_key = os.getenv("OPENAI_API_KEY")

classification_template = PromptTemplate.from_template(
    """Você deve classificar uma pergunta
    
    Dada a pergunta do usuário abaixo, classifique-a como sendo sobre `Carro`, `Restaurante` ou `Tecnologia`.

    <Se a pergunta for sobre mecânica de carros, modelos ou tecnologia automotiva, classifique-a como 'Carro'>
    <Se a pergunta for sobre piadas, classifique-a como 'Piada'>
    <Se a pergunta for sobre patentes, classifique-a como 'Patente'>
    
    <question>
    {question}
    </question>
    
    Classificação:"""
)

classification_chain = classification_template | ChatOpenAI() | StrOutputParser()

In [4]:
import re

carro_template = """Você é um especialista em automóveis. Você tem amplo conhecimento sobre mecânica de carros, \
modelos e tecnologia automotiva. Você fornece respostas claras e úteis sobre carros.\
\
Pergunta:\
{query}"""

piada_template = """Você é um comediante,alegre e divertido. Dada uma pergunta crie uma piada sobre a pergunta \
\
Pergunta:\
{query}"""

patent_template = """Você é um assistente que responde perguntas sobre pedidos de patente, levando em conta o contexto {contexto} \
\
Pergunta:\
{query}"""

# teste expressão regular online https://regex101.com/

def extrair_numero_pedido(texto):
    padrao = r"(PI|MU|C1|C2|C3|C4|C5|C6|C7|C8|C9)\s*\d{7}(?:-\d)?|(?:BR\s*)?(?:\d{2}\s*)?(?:\d{4}\s*)?\d{6}(?:-\d)?"
    match = re.search(padrao, texto)
    if match:
        numero_pedido = match.group()
        numero_pedido = numero_pedido.replace(" ", "").upper().strip()
        return numero_pedido
    else:
        return None
        
def prompt_router(input):
    classification = classification_chain.invoke({"question": input["query"]})
   
    if classification == "Carro":
        print("Questão relativa a carros")
        return PromptTemplate.from_template(carro_template)
    elif classification == "Piada":
        print("Questão relativa a piada")
        return PromptTemplate.from_template(piada_template)
    elif classification == "Patente":
        print("Questão relativa a patentes")
        numero = extrair_numero_pedido(input["query"])
        if numero:
            contexto = f"O pedido {numero} teve carta patente concedida em 2024" 
        else:
            contexto = "Informações adicionais sobre o pedido não foram encontradas."
        return PromptTemplate.from_template(patent_template).format(query=input["query"], contexto=contexto)
    else:
        print("Unexpected classification:", classification)
        return None

query = "Qual o andamento do pedido de patente PI8900908?"
#query = "Qual o andamento do pedido de patente ?"
query = "Quais as características de um veículo sedan ?"
input_query = {"query": query}
prompt = prompt_router(input_query)

Questão relativa a carros


In [5]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

In [6]:
query = "Qual o andamento do pedido de patente PI8900908?"
chain.invoke(query)

Questão relativa a patentes


'O pedido de patente PI8900908 teve a carta patente concedida em 2024, o que significa que o processo de patenteamento foi concluído com sucesso e a invenção foi oficialmente protegida. Se você está procurando informações atualizadas sobre o andamento do pedido de patente, recomendo entrar em contato com o órgão responsável pela concessão de patentes em seu país ou com um advogado especializado em propriedade intelectual. Eles poderão fornecer detalhes sobre qualquer atualização ou status adicional relacionado ao pedido de patente PI8900908.'

In [7]:
chain.invoke("qual o concorrente do Compass ?")

Questão relativa a carros


'O Jeep Compass possui diversos concorrentes no mercado de SUVs compactos, como o Hyundai ix35, o Ford Ecosport, o Honda HR-V, o Chevrolet Tracker, o Nissan Kicks, entre outros. Cada um desses modelos possui suas próprias características e vantagens, sendo importante comparar as especificações, desempenho e preço de cada um antes de tomar uma decisão de compra.'